In [1]:
import nltk 
from nltk import porter, lancaster,pos_tag
from nltk.corpus import stopwords
import os
from afinn import Afinn
import pandas as pd
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import numpy as np
import re
#import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize, RegexpTokenizer
#from sklearn.ensemble import RandomForestRegressor ,RandomForestClassifier
from collections import Counter
import emoji
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91902\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\91902\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
ps = PorterStemmer()

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [5]:
df['sentiment'].value_counts()

1    4311
2    2382
0     456
3     125
Name: sentiment, dtype: int64

In [6]:
df['tweet_id'].nunique()

7274

# #Removing null row from dataset

In [8]:
df.isnull().sum()

tweet_id     0
tweet        1
sentiment    0
dtype: int64

In [9]:
df.shape

(7274, 3)

In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [12]:
df.shape

(7273, 3)

In [13]:
df['tweet'].nunique()

7255

In [14]:
df['tweet'].duplicated().sum()

18

# # Performing basic preprocessing under tweetprocessing()

Lowercase

Punctuation Removal

Emoji detection and removal

Non ASCII characters detection and removal

Unique token stemming words

Basic stopwords removal


In [15]:
def tweetprocessing(df):
    
    def extract_emojis(str):
        str = ''.join(c for c in str if c in emoji.UNICODE_EMOJI)
        return str

    def replace_emoji(str):
        str = ''.join(i for i in str if not i in extract_emojis(str))
        return str
    
    def extract_nonascii(str):
        str = ''.join(c for c in str if  ord(c) > 128)
        return str


    def replace_nonascii(str):
        str = ''.join(i for i in str if not i in extract_nonascii(str))
        return str
    
    def get_unique_tokens_lower_stem(text):
        unique_tokens_lower_stem = set([ps.stem(w) for w in word_tokenize(text)])
        return ' '.join(unique_tokens_lower_stem)
    
    def get_stopwords(text):
        stop_words = set(stopwords.words('english'))
        total_stopwords = set([w for w in word_tokenize(text) if w not in stop_words])
        return ' '.join(total_stopwords)
    
    "__________________________Convert to Lower Case____________________________________"
    df['tweet'] = df['tweet'].str.lower()
    
    "__________________________Remove Punctuations______________________________________"
    
    df['tweet'] = df['tweet'].str.replace('[^\w\s]','')
    
    "__________________________Remove Emoji_____________________________________________"
    df['tweet'] = df['tweet'].apply(lambda x: replace_emoji(x))
    
    
    "__________________________Remove non ASCII_________________________________________"
      
    df['tweet'] = df['tweet'].apply(lambda x: replace_nonascii(x))
    
    
    "__________________________Get unique tokens with stemming__________________________"
    df['unique_tokens_lower_stem'] = df['tweet'].apply(lambda x :get_unique_tokens_lower_stem(x))
    
    
            
    "__________________Remove basic stopwords and get unique words from tweet____________"
    df['no_stopwords'] = df['unique_tokens_lower_stem'].apply(lambda x :get_stopwords(x) )
    
    return df

In [16]:
newdf = tweetprocessing(df)

In [17]:
newdf.head()

,tweet_id,tweet,sentiment,unique_tokens_lower_stem,no_stopwords
0,1701,sxswnui sxsw apple defining language of touch ...,1,defin differ touch dialect appl smaller becom ...,defin touch sxswnui dialect appl smaller becom...
1,1851,learning ab google doodles all doodles should ...,1,sxsw all googledoodl learn for occas except si...,googledoodl ab learn light except signific fun...
2,2689,one of the most inyourface ex of stealing the ...,2,inyourfac most school mention yr the appl expe...,sxsw inyourfac school rt mention yr appl exper...
3,4525,this iphone sxsw app would b pretty awesome if...,0,brows illmakeitwork everi it iphon awesom didn...,brows awesom extend thi didnt illmakeitwork b ...
4,3604,line outside the apple store in austin waiting...,1,line austin ipad wait the for outsid appl link...,line austin ipad wait outsid appl link store n...


# # Creating sentiment scorewise dataframes

In [18]:
def get_sentimentwise_df(df):
    
    df0 = df[df['sentiment']==0]
    df1 = df[df['sentiment']==1]
    df2 = df[df['sentiment']==2]
    df3 = df[df['sentiment']==3]
    return df0,df1,df2,df3

In [19]:
df0,df1,df2,df3 = get_sentimentwise_df(newdf)

In [20]:
df0.head()

,tweet_id,tweet,sentiment,unique_tokens_lower_stem,no_stopwords
3,4525,this iphone sxsw app would b pretty awesome if...,0,brows illmakeitwork everi it iphon awesom didn...,brows awesom extend thi didnt illmakeitwork b ...
24,634,mention i have a 3g iphone after 3 hrs tweetin...,0,mention wa dead it station have hr 3 upgrad ip...,sxsw hr mention tweet dead plugin station 3g r...
34,1178,so i went the whole day wout my laptop amp jus...,0,whole cloudapp laptop ftp so i ipad the my wen...,day miss ipad went whole 1 thing use skype clo...
50,5058,rt mention google lost its way by caring too m...,0,much rt lost the mention care user psych busi ...,much rt lost mention care psych busi googl vs ...
63,2435,i composed a tweet so acerbic and cynical abou...,0,twitterpow phone so i ipad imthatgood that use...,sxsw acerb ipad twitterpow imthatgood tweet du...


In [21]:
df1.head()

,tweet_id,tweet,sentiment,unique_tokens_lower_stem,no_stopwords
0,1701,sxswnui sxsw apple defining language of touch ...,1,defin differ touch dialect appl smaller becom ...,defin touch sxswnui dialect appl smaller becom...
1,1851,learning ab google doodles all doodles should ...,1,sxsw all googledoodl learn for occas except si...,googledoodl ab learn light except signific fun...
4,3604,line outside the apple store in austin waiting...,1,line austin ipad wait the for outsid appl link...,line austin ipad wait outsid appl link store n...
5,966,technews one lone dude awaits ipad 2 at apples...,1,ipad ipad_2 tablet tech_new dude await appl 2 ...,ipad ipad_2 tablet lone tech_new dude await ap...
6,1395,sxsw tips prince npr videos toy shopping with ...,1,princ ipad tip zuckerberg toy link with video ...,ipad princ tip zuckerberg toy link video sxsw ...


In [22]:
df2.head()

,tweet_id,tweet,sentiment,unique_tokens_lower_stem,no_stopwords
2,2689,one of the most inyourface ex of stealing the ...,2,inyourfac most school mention yr the appl expe...,sxsw inyourfac school rt mention yr appl exper...
8,8835,free sxsw sampler on itunes link freemusic,2,freemus on sampler free link sxsw itun,freemus sampler link free sxsw itun
9,883,i think i might go all weekend without seeing ...,2,i all ipad might weekend the case twice same t...,ipad case might weekend twice think go see wit...
11,2369,its official im buying an ipad sxsw elevate,2,im an ipad it offici buy sxsw elev,im ipad offici buy sxsw elev
12,3944,theyre giving away ipad 2s x boxes and books a...,2,2s ipad and box x mention book theyr away give...,2s ipad mention box x book theyr away give tec...


In [23]:
df3.head()

,tweet_id,tweet,sentiment,unique_tokens_lower_stem,no_stopwords
10,6398,rt mention official sxsw app sxsw go bitlyhmii...,3,rt ipad mention go offici android bitlyhmiiga ...,rt ipad mention go offici android bitlyhmiiga ...
28,5368,standing on a long line surrounded by unemploy...,3,am i ipad the appl or a line 5th on techi ave ...,line 5th ipad wait techi appl brooklyn long st...
123,4618,googlebing search smackdown panel is in a gian...,3,room panel have giant googleb hope a you is fo...,giant googleb chair room quotfloorsittingquot ...
133,7469,original products for 1 device is nuts sxsw eg...,3,ipad eg devic the origin 1 is nut for product ...,ipad eg devic origin 1 nut product sxsw
242,2116,iphone crashed in front of sxsw apple popup be...,3,popup front crash appl in bestworstthingev of ...,popup crash front appl bestworstthingev sxsw i...
